In [1]:
import os
import sys

sys.path.append('../')

In [2]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

In [3]:
from benchmark.data import read_tep_data
from benchmark.deep_learning import TimeSeriesClassifier
from benchmark.deep_learning import FullyConvolutionalEncoder

## Reading the data

In [4]:
src, tgt = 1, 2

In [5]:
Xs, ys = read_tep_data(
    base_path='../tep_data/benchmark/',
    modes=[src,],
    normalization="standardization",
    return_domain=False,
    as_list=False,
    channels_first=True,
    one_hot_labels=False
)

In [6]:
Xs.shape, ys.shape

(torch.Size([2900, 34, 600]), torch.Size([2900]))

In [7]:
Xt, yt = read_tep_data(
    base_path='../tep_data/benchmark/',
    modes=[tgt,],
    normalization="standardization",
    return_domain=False,
    as_list=False,
    channels_first=True,
    one_hot_labels=False
)

In [8]:
Xt.shape, yt.shape

(torch.Size([2845, 34, 600]), torch.Size([2845]))

In [9]:
src_dataset = torch.utils.data.TensorDataset(Xs, ys)
src_dataloader = torch.utils.data.DataLoader(
    src_dataset, batch_size=256, shuffle=True)

tgt_dataset = torch.utils.data.TensorDataset(Xt, yt)
tgt_dataloader = torch.utils.data.DataLoader(
    tgt_dataset, batch_size=256, shuffle=True)

## Creating the neural net

In [10]:
encoder = FullyConvolutionalEncoder(
    n_features=Xs.shape[1],
    n_time_steps=Xs.shape[2],
    batch_norm=False,
    instance_norm=True
)

In [11]:
clf = torch.nn.Linear(128, 29)

In [12]:
n_epochs = 70
momentum = 0.9
l2_penalty = 0.0
learning_rate = 1e-4
optimizer_name = 'adam'

In [13]:
model = TimeSeriesClassifier(encoder=encoder,
                             clf=clf,
                             n_classes=29,
                             loss_fn=torch.nn.CrossEntropyLoss(),
                             input_shape=(Xs.shape[1:]),
                             learning_rate=learning_rate,
                             l2_penalty=l2_penalty,
                             momentum=momentum,
                             optimizer_name=optimizer_name)

## Training with Pytorch Lightning

In [14]:
log_dir = f"../models/logs/{src}->{tgt}"
log_name = "single_source_only_baseline"

In [15]:
logger = TensorBoardLogger(save_dir=log_dir, name=log_name)

In [16]:
trainer = pl.Trainer(
    max_epochs=n_epochs,
    accelerator='gpu',
    logger=logger,
    enable_checkpointing=True,
    enable_progress_bar=True)
trainer.fit(model,
            src_dataloader,
            tgt_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                      | Params | In sizes      | Out sizes
----------------------------------------------------------------------------------
0 | clf     | Linear                    | 3.7 K  | [16, 128]     | [16, 29] 
1 | encoder | FullyConvolutionalEncoder | 301 K  | [16, 34, 600] | [16, 128]
2 | loss_fn | CrossEntropyLoss          | 0      | ?             | ?        
----------------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

/home/efernand/anaconda3/envs/OptimalTransport/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/home/efernand/anaconda3/envs/OptimalTransport/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/efernand/anaconda3/envs/OptimalTransport/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=70` reached.
